In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

class_name = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']


In [5]:
from torchvision import datasets,transforms
data_path = '../data_unversioned/p1ch7/'
cifar10 =datasets.CIFAR10(data_path,train=True,download=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize(
                                  (0.4915,0.4823,0.4468),
                                  (0.2470,0.2435,0.2616))
                          ]))

Files already downloaded and verified


In [6]:
cifar10_val = datasets.CIFAR10(
    data_path,train=False,download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915,0.4823,0.4468),
                             (0.2470,0.2435,0.2616))
    ])
)

Files already downloaded and verified


In [7]:
label_map = {0:0,2:1}
class_names = ['airplane','bird']
cifar2 = [(img,label_map[label]) for img,label in cifar10 if label in [0,2]]
cifar2_val = [(img,label_map[label]) for img,label in cifar10_val if label in [0,2]]

In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,16,kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(16,8,kernel_size=3,padding=1)
        self.fc1 =  nn.Linear(8*8*8,32)
        self.fc2 = nn.Linear(32,2)

    def forward(self,x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)),2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)),2)
        out = out.view(-1,8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [9]:
model = Net()

numel_list = [p.numel() for p in model.parameters()]
print(sum(numel_list),numel_list)

18090 [432, 16, 1152, 8, 16384, 32, 64, 2]


In [13]:
import datetime

def training_loop_gpu(n_epochs,model,optimizer,loss_fn,train_loader,val_loader):
    for epoch in range(1,n_epochs+1):
        loss_train=0.0
        loss_val =0.0

        for imgs,labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs,labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        optimizer.zero_grad()
        for imgs_val,labels_val in val_loader:
            imgs_val = imgs_val.to(device=device)
            labels_val = labels_val.to(device= device)
            outputs_val = model(imgs_val)
            loss = loss_fn(outputs_val,labels_val)

            loss_val += loss.item()

        if epoch ==1 or epoch%10 ==0:
            print('{}  Epoch:{}  Loss train:{}  Val loss:{}'.format(datetime.datetime.now(),epoch,loss_train/len(train_loader),loss_val/len(val_loader)))


In [11]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {device}")

Training on device cuda


In [12]:
model = Net()
model = model.to(device=device)
optimizer = optim.SGD(model.parameters(),lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(cifar2,batch_size=64,shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar2_val,batch_size = 64,shuffle=True)

training_loop_gpu(n_epochs=100,model=model,optimizer=optimizer,loss_fn=loss_fn,train_loader=train_loader,val_loader=val_loader)

2023-06-27 10:59:13.310930  Epoch:2  Loss train:0.5606666757802296  Val loss:0.48174979723989964
2023-06-27 10:59:15.551796  Epoch:11  Loss train:0.33611617014286627  Val loss:0.33389222901314497
2023-06-27 10:59:18.048039  Epoch:21  Loss train:0.2995045527720907  Val loss:0.33052949979901314
2023-06-27 10:59:20.554094  Epoch:31  Loss train:0.27394585852410386  Val loss:0.31313083693385124
2023-06-27 10:59:23.068774  Epoch:41  Loss train:0.24962039486427975  Val loss:0.2741107544861734
2023-06-27 10:59:25.603272  Epoch:51  Loss train:0.22359255079630833  Val loss:0.2771421354264021
2023-06-27 10:59:28.344768  Epoch:61  Loss train:0.205898967945272  Val loss:0.28140347357839346
2023-06-27 10:59:30.696196  Epoch:71  Loss train:0.18642208331329807  Val loss:0.27881104685366154
2023-06-27 10:59:33.164879  Epoch:81  Loss train:0.17853569111246972  Val loss:0.29058569204062223
2023-06-27 10:59:35.582568  Epoch:91  Loss train:0.16088399886610402  Val loss:0.26365976594388485
2023-06-27 10:59:

In [17]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,16,kernel_size=5,padding=2)
        self.conv2 = nn.Conv2d(16,8,kernel_size=3,padding=1)
        self.fc1 =  nn.Linear(8*8*8,32)
        self.fc2 = nn.Linear(32,2)

    def forward(self,x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)),2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)),2)
        out = out.view(-1,8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [18]:
model = Net()

numel_list = [p.numel() for p in model.parameters()]
print(sum(numel_list),numel_list)

18858 [1200, 16, 1152, 8, 16384, 32, 64, 2]


In [19]:
model = Net()
model = model.to(device=device)
optimizer = optim.SGD(model.parameters(),lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(cifar2,batch_size=64,shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar2_val,batch_size = 64,shuffle=True)

training_loop_gpu(n_epochs=100,model=model,optimizer=optimizer,loss_fn=loss_fn,train_loader=train_loader,val_loader=val_loader)

2023-06-27 11:06:47.373630  Epoch:1  Loss train:0.5768503905481593  Val loss:0.4980163900181651
2023-06-27 11:06:50.110050  Epoch:10  Loss train:0.32538754772988093  Val loss:0.3217166452668607
2023-06-27 11:06:53.056384  Epoch:20  Loss train:0.2885088966150952  Val loss:0.330152902752161
2023-06-27 11:06:55.752876  Epoch:30  Loss train:0.2654997815561902  Val loss:0.3057402675040066
2023-06-27 11:06:58.432750  Epoch:40  Loss train:0.2449848030687897  Val loss:0.30388079211115837
2023-06-27 11:07:01.131551  Epoch:50  Loss train:0.22501578435871253  Val loss:0.30546115012839437
2023-06-27 11:07:03.846720  Epoch:60  Loss train:0.21018418437166578  Val loss:0.2751196031458676
2023-06-27 11:07:06.564555  Epoch:70  Loss train:0.1936900867683113  Val loss:0.28276640037074685
2023-06-27 11:07:09.362403  Epoch:80  Loss train:0.17565738950755186  Val loss:0.3022459000349045
2023-06-27 11:07:12.097761  Epoch:90  Loss train:0.16157266115591784  Val loss:0.2710900856181979
2023-06-27 11:07:14.8183

In [25]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,16,kernel_size=(3,1),padding=1)#32*32 -> 32*34
        self.tanh1 = nn.Tanh()
        self.maxpool1 = nn.MaxPool2d(2)#16*17
        self.conv2 = nn.Conv2d(16,8,kernel_size=(3,1),padding=1) #16*19
        self.tanh2 = nn.Tanh()
        self.maxpool2 = nn.MaxPool2d(2) #8*9
        self.fc1 = nn.Linear(8*8*9,32)
        self.tanh3 = nn.Tanh()
        self.fc2 = nn.Linear(32,2)

    def forward(self,x):
        out = self.maxpool1(self.tanh1(self.conv1(x)))
        out = self.maxpool2(self.tanh2(self.conv2(out)))
        out = out.view(-1,8*9*8)
        out = self.tanh3(self.fc1(out))
        out = self.fc2(out)
        return out

In [26]:
model = Net()
model = model.to(device=device)
optimizer = optim.SGD(model.parameters(),lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(cifar2,batch_size=64,shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar2_val,batch_size = 64,shuffle=True)

training_loop_gpu(n_epochs=100,model=model,optimizer=optimizer,loss_fn=loss_fn,train_loader=train_loader,val_loader=val_loader)

2023-06-27 11:28:35.525352  Epoch:1  Loss train:0.6564535403707225  Val loss:0.593817014247179
2023-06-27 11:28:38.333285  Epoch:10  Loss train:0.3714986274576491  Val loss:0.3706735325977206
2023-06-27 11:28:40.946539  Epoch:20  Loss train:0.3175392764009488  Val loss:0.329670743085444
2023-06-27 11:28:43.374341  Epoch:30  Loss train:0.29643475544300807  Val loss:0.3425480271689594
2023-06-27 11:28:45.853938  Epoch:40  Loss train:0.27766544817929056  Val loss:0.3068947782739997
2023-06-27 11:28:48.400143  Epoch:50  Loss train:0.2628451426317737  Val loss:0.3146332479082048
2023-06-27 11:28:50.811755  Epoch:60  Loss train:0.2503147942909769  Val loss:0.30217039259150624
2023-06-27 11:28:53.202978  Epoch:70  Loss train:0.2405147137725429  Val loss:0.2870688480325043
2023-06-27 11:28:55.608185  Epoch:80  Loss train:0.22999016480271223  Val loss:0.3112298958003521
2023-06-27 11:28:58.011793  Epoch:90  Loss train:0.21874393100381656  Val loss:0.3003285499289632
2023-06-27 11:29:00.461211  

In [27]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,16,kernel_size=(3,3),padding=1)
        self.tanh1 = nn.Tanh()
        self.maxpool1 = nn.MaxPool2d(2)#16*17
        self.conv2 = nn.Conv2d(16,8,kernel_size=(3,3),padding=1)
        self.tanh2 = nn.Tanh()
        self.maxpool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8*8*8,32)
        self.tanh3 = nn.Tanh()
        self.fc2 = nn.Linear(32,2)

    def forward(self,x):
        out = self.maxpool1(self.tanh1(self.conv1(x)))
        out = self.maxpool2(self.tanh2(self.conv2(out)))
        out = out.view(-1,8*8*8)
        out = self.tanh3(self.fc1(out))
        out = self.fc2(out)
        return out

In [28]:
cifar_out = [(img,label) for img,label in cifar10 if label not in [0,2]]

In [30]:
for i,(img,label) in enumerate(cifar_out):
    if i% 100 == 0 :print('progression',i)
    img = img.unsqueeze(0).to(device=device)
    m = nn.Softmax(dim=1).to(device=device)
    prediction = m(model(img))

    if prediction[0][0]>0.95 or prediction[0][0]<0.5:
        print(i,label,prediction)
        break

progression 0
0 6 tensor([[0.1950, 0.8050]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
